In [1]:
import pickle 
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
import numpy as np
import re
import librosa
import string
import matplotlib.pyplot as plt
import os
import noisereduce as nr
import IPython
from collections import Counter
from sklearn.metrics import mean_squared_error, accuracy_score

torch.manual_seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(10)

In [34]:
from torch.utils.tensorboard import SummaryWriter



In [3]:
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')

In [4]:
def setlr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

def lr_decay(optimizer, epoch, learning_rate):
    if epoch%10==0:
        new_lr = learning_rate / (10**(epoch//10))
        optimizer = setlr(optimizer, new_lr)
        print("Changed learning rate to {}".format(new_lr))
    return optimizer

In [5]:
BASE = '/data/home/advaitmb/datasets/sentences/'
vocab_size = 2255

In [6]:
import pandas as pd
audio = pd.read_csv('audio_df_improvised.csv')

audio.reset_index(inplace=True)
audio.emotion = pd.Categorical(pd.factorize(audio.emotion)[0])
audio.head()

,index,start_time,end_time,wav_file,emotion,val,act,dom
0,0,6.2901,8.2357,Ses01F_impro01_F000,0,2.5,2.5,2.5
1,1,10.0100,11.3925,Ses01F_impro01_F001,0,2.5,2.5,2.5
2,2,14.8872,18.0175,Ses01F_impro01_F002,0,2.5,2.5,2.5
3,3,27.4600,31.4900,Ses01F_impro01_F005,0,2.5,3.5,2.0
4,4,85.2700,88.0200,Ses01F_impro01_F012,1,2.0,3.5,3.5


In [7]:
# Generate Spectrogram and Normalize 

def get_melspectrogram_db(file_path, sr=None, n_fft=2048, hop_length=512, n_mels=128,top_db=80):
    wav,sr = librosa.load(file_path,sr=sr)
    
    if wav.shape[0]<5*sr:
        wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:5*sr]
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels)
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    return spec_db

def normalize(spec):
    return (spec - spec.mean())/(spec.std())

In [8]:
def generate_spectrograms(base, df, in_col, out_col):
    data = []
    labels = []
    for ind in tqdm(range(len(df))):
        row = audio.iloc[ind]
        file_path = base + row[in_col] + '.wav'
        data.append(normalize(get_melspectrogram_db(file_path))[np.newaxis,...])
        labels.append(row[out_col])
    return (data, labels)

In [13]:
# Generate Pickel Files 
import pickle

spec = generate_spectrograms(BASE, audio, 'wav_file', 'emotion')

with open('spectrograms.pkl', 'wb') as handle:
    pickle.dump(spec, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 2943/2943 [00:59<00:00, 49.14it/s]


In [46]:
with open('spectrograms.pkl', 'rb') as handle:
    (data, labels) = pickle.load(handle)

In [47]:
data = np.array(data)
labels = np.array(labels)

In [48]:
data.shape

(2943, 1, 128, 157)

In [49]:
class Audio(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

Text Setup

In [7]:
old_text = pd.read_csv('text_df.csv')
old_text.emotion = audio.emotion
old_text.emotion = pd.Categorical(pd.factorize(old_text.emotion)[0])
old_text.head()

,wav_file,emotion,transcription
0,Ses01F_impro01_F000,0,excuse me .
1,Ses01F_impro01_F001,0,yeah .
2,Ses01F_impro01_F002,0,is there a problem ?
3,Ses01F_impro01_F005,0,well what s the problem ? let me change it .
4,Ses01F_impro01_F012,1,that s out of control .


In [8]:
text = pd.DataFrame()
text['data'] = old_text.transcription
text['label'] = old_text.emotion

text.head()

,data,label
0,excuse me .,0
1,yeah .,0
2,is there a problem ?,0
3,well what s the problem ? let me change it .,0
4,that s out of control .,1


In [9]:
#tokenization
import spacy
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]


In [10]:
#count number of occurences of each word
counts = Counter()
for index, row in text.iterrows():
    counts.update(tokenize(row['data']))
    
    
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [11]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [12]:
text['encoded_data'] = text['data'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
text.head()

,data,label,encoded_data
0,excuse me .,0,"[[2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,yeah .,0,"[[5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,is there a problem ?,0,"[[6, 7, 8, 9, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,well what s the problem ? let me change it .,0,"[[10, 11, 12, 13, 9, 14, 15, 3, 16, 17, 4, 0, ..."
4,that s out of control .,1,"[[18, 12, 19, 20, 21, 4, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
text['encoded_data'][16]

array([array([84, 45, 85, 33, 11, 42, 86,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0]),
       8], dtype=object)

In [69]:
text_encodings = (X, y)
# # Generate Pickel Files 
# import pickle

with open('text_encodings.pkl', 'wb') as handle:
    pickle.dump(text_encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
with open('text_encodings.pkl', 'rb') as handle:
    (X, y) = pickle.load(handle)
    
X = np.array(X)
y = np.array(y)

In [13]:
X = list(text['encoded_data'])
y = list(text['label'])
X = np.array(X)
y = np.array(y)
# from sklearn.model_selection import train_test_split
# X_train = X[:2214]
# X_valid = X[2214:]
# y_train = y[:2214]
# y_valid = y[2214:]



In [14]:
class Text(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

2255

In [86]:

train_text = Text(X_train, y_train)
valid_text = Text(X_valid, y_valid)



batch_size = 1000
text_train_loader = DataLoader(train_text, batch_size=batch_size, shuffle=False)
text_valid_loader = DataLoader(valid_text, batch_size=batch_size, shuffle=False)

In [15]:
def load_glove_vectors(glove_file=".vector_cache/glove.6B.50d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors

In [16]:
def get_emb_matrix(pretrained, word_counts, emb_size = 50):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_counts) + 2
    vocab_to_idx = {}
    vocab = ["", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_counts:
        if word in word_vecs:
            W[i] = word_vecs[word]
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, np.array(vocab), vocab_to_idx

In [17]:
word_vecs = load_glove_vectors()


In [18]:
pretrained_weights, vocab, vocab2index = get_emb_matrix(word_vecs, counts)

In [19]:
class LSTM(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = True 
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 4)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [20]:
lstm = LSTM(vocab_size, 50, 64, pretrained_weights)
lstm.to(device)

LSTM(
  (embeddings): Embedding(2255, 50, padding_idx=0)
  (lstm): LSTM(50, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=4, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [50]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [36]:
def train_text_model(model, loss_fn, train_loader, valid_loader, epochs, learning_rate, optimizer, train_losses, valid_losses, comment, change_lr=None):
    print("Training Text Model")
#     tb = SummaryWriter(comment=comment)
    for epoch in range(1, epochs+1):
        model.train()
        batch_losses=[]
        if change_lr and epoch != 0:
            optimizer = change_lr(optimizer, epoch, learning_rate)
        for x, y, l in text_train_loader:
            x = x.to(device, dtype=torch.long)
            y = y.to(device, dtype=torch.long)
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())
            y = y.cpu().detach()
        train_losses.append(batch_losses)
            
        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []
        for x, y, l in text_valid_loader:
            x = x.to(device, dtype=torch.long)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x, l)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())

        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
        unweighted_accuracy = accuracy_score( trace_yhat.argmax(axis=1), trace_y )
        weighted_accuracy = balanced_accuracy_score( trace_yhat.argmax(axis=1), trace_y )
        
#         tb.add_scalar(
#             'train_loss', np.mean(train_losses[-1]), epoch
#         )
#         tb.add_scalar(
#             'valid_loss', np.mean(valid_losses[-1]), epoch
#         )
#         tb.add_scalar(
#             'Unweighted Accuracy', unweighted_accuracy, epoch
#         )     
#         tb.add_scalar(
#             'Weighted Accuracy', weighted_accuracy, epoch
#         )   
        if epoch%5 == 0:
            print("Epoch - {} Train-Loss : {} Valid-Loss : {}".format(epoch, np.mean(train_losses[-1]), np.mean(valid_losses[-1])))
            print("unweighted_accuracy : {} weighted_accuracy : {}".format(unweighted_accuracy, weighted_accuracy ))
#         print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))
    return unweighted_accuracy, weighted_accuracy

In [95]:
learning_rate = 0.01
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)
epochs = 300
loss_fn = nn.CrossEntropyLoss()
train_losses=[]
valid_losses=[]

In [101]:
train_text_model(lstm, loss_fn, text_train_loader, text_valid_loader, epochs, learning_rate, optimizer, train_losses, valid_losses, "text validation 5" ,None)

Training Text Model
Epoch - 5 Train-Loss : 1.4000828266143799 Valid-Loss : 1.4073214530944824
unweighted_accuracy : 0.18244170096021947 weighted_accuracy : 0.18244170096021947
Epoch - 10 Train-Loss : 1.4000670909881592 Valid-Loss : 1.4073214530944824
unweighted_accuracy : 0.18244170096021947 weighted_accuracy : 0.18244170096021947
Epoch - 15 Train-Loss : 1.4000750382741292 Valid-Loss : 1.4073214530944824
unweighted_accuracy : 0.18244170096021947 weighted_accuracy : 0.18244170096021947
Epoch - 20 Train-Loss : 1.4001056750615437 Valid-Loss : 1.4073214530944824
unweighted_accuracy : 0.18244170096021947 weighted_accuracy : 0.18244170096021947
Epoch - 25 Train-Loss : 1.4000577926635742 Valid-Loss : 1.4073214530944824
unweighted_accuracy : 0.18244170096021947 weighted_accuracy : 0.18244170096021947
Epoch - 30 Train-Loss : 1.399972677230835 Valid-Loss : 1.4073214530944824
unweighted_accuracy : 0.18244170096021947 weighted_accuracy : 0.18244170096021947
Epoch - 35 Train-Loss : 1.40002759297688

KeyboardInterrupt: 

5 Fold Cross Validation

In [57]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

def train_audio_model(model, loss_fn, train_loader, valid_loader, epochs, learning_rate, optimizer, train_losses, valid_losses, comment, change_lr=None):
    print("Training Audio Model")
#     tb = SummaryWriter(comment=comment)
    for epoch in range(1, epochs+1):
        
        model.train()
        batch_losses=[]
        if change_lr:
            optimizer = change_lr(optimizer, epoch, learning_rate)
        for i, data in tqdm(enumerate(train_loader)):
            x, y = data
            optimizer.zero_grad()
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            batch_losses.append(loss.item())
            optimizer.step()
        train_losses.append(batch_losses)


#         print("Epoch - {} Train-Loss : {}".format(epoch, np.mean(train_losses[-1])))
        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []
        for i, data in enumerate(valid_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())
        
        

        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
        unweighted_accuracy = accuracy_score( trace_yhat.argmax(axis=1), trace_y )
        weighted_accuracy = balanced_accuracy_score( trace_yhat.argmax(axis=1), trace_y )
        
        
        
#         tb.add_scalar(
#             'train_loss', np.mean(train_losses[-1]), epoch
#         )
#         tb.add_scalar(
#             'valid_loss', np.mean(valid_losses[-1]), epoch
#         )
#         tb.add_scalar(
#             'Unweighted Accuracy', unweighted_accuracy, epoch
#         )     
#         tb.add_scalar(
#             'Weighted Accuracy', weighted_accuracy, epoch
#         )   
        
        print("Epoch - {} Train-Loss : {} Valid-Loss : {}".format(epoch, np.mean(train_losses[-1]), np.mean(valid_losses[-1])))
        print("unweighted_accuracy : {} weighted_accuracy : {}".format(unweighted_accuracy, weighted_accuracy ))
    return unweighted_accuracy, weighted_accuracy 

In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5, 2, padding=1) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, 2, padding=1) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 5, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 5, 2, padding=1)
        
        self.fc1 = nn.Linear((7*8*128), 512) #flattening.
        
        
        self.fc2 = nn.Linear(512, 4) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).
        self.activation = nn.Softmax(dim=1)

    def convs(self, x):
        # max pooling over 2x2
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        x = F.relu(self.conv4(x))

        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return self.activation(x)


In [58]:
def validate_audio(train_loader, valid_loader, comment):
    net = CNN()
    net.to(device)
    learning_rate = 2e-5
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    epochs = 15
    loss_fn = nn.CrossEntropyLoss()
    train_losses=[]
    valid_losses=[]
    valid_accuracy = []
    uwa, wa = train_audio_model(net, loss_fn, train_loader, valid_loader, epochs, learning_rate, optimizer, train_losses, valid_losses, comment,change_lr=lr_decay)
    print("validation over")
    torch.save(net, comment+'.pth')
    del net
    
    return np.mean(valid_losses[-1]), uwa, wa  

In [37]:
def validate_text(train_loader, valid_loader, vocab_size, embedding_dim, hidden_dim, pretrained_weights, comment):
    net = LSTM(vocab_size, embedding_dim, hidden_dim, pretrained_weights)
    net.to(device)
    learning_rate = 0.01
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    epochs = 300
    loss_fn = nn.CrossEntropyLoss()
    train_losses=[]
    valid_losses=[]
    valid_accuracy = []
    uwa, wa = train_text_model(net, loss_fn, train_loader, valid_loader, epochs, learning_rate, optimizer, train_losses, valid_losses, comment, None)
    print("validation over")
    torch.save(net, comment+'.pth')
    del net
    
    return np.mean(valid_losses[-1]), uwa, wa  

In [59]:
# 0-242 1F
# 243-522 1M
# 522-773 2F
# 773-1052 2M
# 1053 - 1314 3F
# 1315 - 1679 3M
# 1680 - 1992 4F
# 1993 - 2213 4M
# 2214 - 2616 5F
# 2617 : 5M
ses_idx = np.array([ range(0,523), range(523, 1052), range(1052, 1680), range(1680, 2214), range(2214, len(audio)) ])
# ses_idx = np.array([range(0, 243), range(243, 523), range(523, 773), range(773, 1052), range(1052, 1315), range(1315, 1680), range(1680, 1993), range(1993, 2214), range(2214, 2617), range(2617,len(audio))])
    

In [60]:
train_audio = []
valid_audio = []
train_text = []
valid_text = []

for i in range(ses_idx.shape[0]):
    train_audio.append(Audio(np.delete(data,ses_idx[i], axis=0), np.delete(labels,ses_idx[i], axis=0)))
    valid_audio.append(Audio(data[ses_idx[i]], labels[ses_idx[i]]))
    
    train_text.append(Text(np.delete(X,ses_idx[i], axis=0), np.delete(y,ses_idx[i], axis=0)))
    valid_text.append(Text(X[ses_idx[i]], y[ses_idx[i]]))
        
    

In [61]:
a_losses = []
a_uwas = []
a_was = []

In [62]:
audio_train_loader = DataLoader(train_audio[0], batch_size=32, shuffle=False)
audio_valid_loader = DataLoader(valid_audio[0], batch_size=32, shuffle=False)
comment="Audio validation : {}".format(0)
a_loss, a_uwa, a_wa = validate_audio(audio_train_loader, audio_valid_loader, comment)

a_losses.append(a_loss)
a_uwas.append(a_uwa)
a_was.append(a_wa)


1it [00:00,  7.35it/s]

Training Audio Model


76it [00:10,  7.32it/s]
1it [00:00,  7.64it/s]

Epoch - 1 Train-Loss : 1.3324374462428845 Valid-Loss : 1.273698883898118
unweighted_accuracy : 0.5353728489483748 weighted_accuracy : 0.5277777777777778


76it [00:10,  7.43it/s]
1it [00:00,  7.74it/s]

...

76it [00:10,  7.44it/s]
1it [00:00,  6.37it/s]

Epoch - 14 Train-Loss : 0.994466013030002 Valid-Loss : 1.1482669044943417
unweighted_accuracy : 0.5965583173996176 weighted_accuracy : 0.5929575422715962


76it [00:10,  7.43it/s]


Epoch - 15 Train-Loss : 0.9919444220630746 Valid-Loss : 1.1476389485246994
unweighted_accuracy : 0.6003824091778203 weighted_accuracy : 0.5972320710823603
validation over


/anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [63]:
audio_train_loader = DataLoader(train_audio[1], batch_size=32, shuffle=False)
audio_valid_loader = DataLoader(valid_audio[1], batch_size=32, shuffle=False)
comment="Audio validation : {}".format(1)
a_loss, a_uwa, a_wa = validate_audio(audio_train_loader, audio_valid_loader, comment)

a_losses.append(a_loss)
a_uwas.append(a_uwa)
a_was.append(a_wa)



1it [00:00,  7.45it/s]

Training Audio Model


76it [00:10,  7.28it/s]
1it [00:00,  7.55it/s]

Epoch - 1 Train-Loss : 1.3349689041313373 Valid-Loss : 1.253240704536438
unweighted_accuracy : 0.48582230623818523 weighted_accuracy : 0.4805913348946136


76it [00:10,  7.18it/s]
1it [00:00,  7.51it/s]

...

Epoch - 13 Train-Loss : 1.0039869311608767 Valid-Loss : 1.0885004681699417
unweighted_accuracy : 0.665406427221172 weighted_accuracy : 0.7003719577044789


76it [00:10,  7.31it/s]
1it [00:00,  7.45it/s]

Epoch - 14 Train-Loss : 1.0013812576469623 Valid-Loss : 1.0887493876849903
unweighted_accuracy : 0.6635160680529301 weighted_accuracy : 0.6964648889387693


76it [00:10,  7.36it/s]


Epoch - 15 Train-Loss : 0.9988806020272406 Valid-Loss : 1.0889782800393946
unweighted_accuracy : 0.665406427221172 weighted_accuracy : 0.6978475558605964
validation over


In [64]:
audio_train_loader = DataLoader(train_audio[2], batch_size=32, shuffle=False)
audio_valid_loader = DataLoader(valid_audio[2], batch_size=32, shuffle=False)
comment="Audio validation : {}".format(2)
a_loss, a_uwa, a_wa = validate_audio(audio_train_loader, audio_valid_loader, comment)

a_losses.append(a_loss)
a_uwas.append(a_uwa)
a_was.append(a_wa)



1it [00:00,  7.43it/s]

Training Audio Model


73it [00:09,  7.32it/s]
1it [00:00,  7.28it/s]

Epoch - 1 Train-Loss : 1.3269077163853058 Valid-Loss : 1.3580786824226379
unweighted_accuracy : 0.3614649681528662 weighted_accuracy : 0.3659274634039764


73it [00:10,  7.29it/s]
1it [00:00,  7.43it/s]

...

Epoch - 13 Train-Loss : 0.9787433833292086 Valid-Loss : 1.1756538182497025
unweighted_accuracy : 0.5843949044585988 weighted_accuracy : 0.5974561645841242


73it [00:10,  7.24it/s]
1it [00:00,  6.08it/s]

Epoch - 14 Train-Loss : 0.9755980225458537 Valid-Loss : 1.1760825037956237
unweighted_accuracy : 0.5828025477707006 weighted_accuracy : 0.595247865691689


73it [00:10,  7.22it/s]


Epoch - 15 Train-Loss : 0.9726241467750236 Valid-Loss : 1.1764829874038696
unweighted_accuracy : 0.5843949044585988 weighted_accuracy : 0.5961663288982877
validation over


In [ ]:
audio_train_loader = DataLoader(train_audio[3], batch_size=32, shuffle=False)
audio_valid_loader = DataLoader(valid_audio[3], batch_size=32, shuffle=False)
comment="Audio validation : {}".format(3)
a_loss, a_uwa, a_wa = validate_audio(audio_train_loader, audio_valid_loader, comment)

a_losses.append(a_loss)
a_uwas.append(a_uwa)
a_was.append(a_wa)



1it [00:00,  7.66it/s]

Training Audio Model


76it [00:10,  7.35it/s]
1it [00:00,  7.32it/s]

Epoch - 1 Train-Loss : 1.3335283116290444 Valid-Loss : 1.3065304545795215
unweighted_accuracy : 0.4232209737827715 weighted_accuracy : 0.4388509316770186


76it [00:10,  7.34it/s]
1it [00:00,  7.31it/s]

...

76it [00:10,  7.35it/s]
1it [00:00,  7.41it/s]

Epoch - 9 Train-Loss : 0.9932330274268201 Valid-Loss : 1.2300321915570427
unweighted_accuracy : 0.4794007490636704 weighted_accuracy : 0.5786065786065787
Changed learning rate to 2.0000000000000003e-06


76it [00:10,  7.37it/s]
1it [00:00,  7.41it/s]

Epoch - 10 Train-Loss : 0.9754643769640672 Valid-Loss : 1.2189198522006763
unweighted_accuracy : 0.4868913857677903 weighted_accuracy : 0.5898828431475668


4it [00:00,  6.91it/s]

In [66]:
audio_train_loader = DataLoader(train_audio[4], batch_size=32, shuffle=False)
audio_valid_loader = DataLoader(valid_audio[4], batch_size=32, shuffle=False)
comment="Audio validation : {}".format(4)
a_loss, a_uwa, a_wa = validate_audio(audio_train_loader, audio_valid_loader, comment)

a_losses.append(a_loss)
a_uwas.append(a_uwa)
a_was.append(a_wa)



70it [00:09,  7.40it/s]
1it [00:00,  7.44it/s]

Epoch - 5 Train-Loss : 1.1211950940745217 Valid-Loss : 1.19667677257372
unweighted_accuracy : 0.541838134430727 weighted_accuracy : 0.6041201173944537


70it [00:09,  7.38it/s]
1it [00:00,  7.18it/s]

Epoch - 6 Train-Loss : 1.092919683456421 Valid-Loss : 1.1941716178603794
unweighted_accuracy : 0.541838134430727 weighted_accuracy : 0.6160096818810512


70it [00:09,  7.30it/s]
1it [00:00,  7.44it/s]

...

Epoch - 13 Train-Loss : 0.9937410524913243 Valid-Loss : 1.163814928220666
unweighted_accuracy : 0.5692729766803841 weighted_accuracy : 0.6241440906817346


70it [00:09,  7.32it/s]
1it [00:00,  7.27it/s]

Epoch - 14 Train-Loss : 0.9913747710841042 Valid-Loss : 1.1637188973634138
unweighted_accuracy : 0.5692729766803841 weighted_accuracy : 0.6236747308110512


70it [00:09,  7.33it/s]


Epoch - 15 Train-Loss : 0.9890804409980773 Valid-Loss : 1.1636328645374463
unweighted_accuracy : 0.5679012345679012 weighted_accuracy : 0.6218800386264457
validation over


In [67]:
mean(a_losses), mean(a_uwas), mean(a_was)

(1.159422809882542, 0.5806207404034505, 0.6198180167113558)

In [ ]:
t_losses = []
t_uwas = []
t_was = []

In [38]:
text_train_loader = DataLoader(train_text[0], batch_size=1000, shuffle=False)
text_valid_loader = DataLoader(valid_text[0], batch_size=1000, shuffle=False)
comment="Text validation : {}".format(0)
t_loss, t_uwa, t_wa = validate_text(text_train_loader, text_valid_loader, vocab_size, 50, 64, pretrained_weights, comment)

t_losses.append(t_loss)
t_uwas.append(t_uwa)
t_was.append(t_wa)

Training Text Model


/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


Epoch - 5 Train-Loss : 1.252090573310852 Valid-Loss : 1.3160866498947144
unweighted_accuracy : 0.42829827915869984 weighted_accuracy : 0.42829827915869984
Epoch - 10 Train-Loss : 1.2540449301401775 Valid-Loss : 1.3165521621704102
unweighted_accuracy : 0.42829827915869984 weighted_accuracy : 0.21455938697318008
Epoch - 15 Train-Loss : 1.2507102092107136 Valid-Loss : 1.321996808052063
unweighted_accuracy : 0.4321223709369025 weighted_accuracy : 0.588310854206808
Epoch - 20 Train-Loss : 1.2536580562591553 Valid-Loss : 1.317845344543457
unweighted_accuracy : 0.42829827915869984 weighted_accuracy : 0.42829827915869984
Epoch - 25 Train-Loss : 1.253665804862976 Valid-Loss : 1.3197951316833496
unweighted_accuracy : 0.42829827915869984 weighted_accuracy : 0.42829827915869984
Epoch - 30 Train-Loss : 1.2531375487645466 Valid-Loss : 1.3199560642242432
unweighted_accuracy : 0.42829827915869984 weighted_accuracy : 0.42829827915869984
Epoch - 35 Train-Loss : 1.2528282006581624 Valid-Loss : 1.31981515

Epoch - 270 Train-Loss : 0.19684582451979318 Valid-Loss : 2.2611701488494873
unweighted_accuracy : 0.4684512428298279 weighted_accuracy : 0.4175217129638099
Epoch - 275 Train-Loss : 0.19920291503270468 Valid-Loss : 2.2720651626586914
unweighted_accuracy : 0.4378585086042065 weighted_accuracy : 0.4116877602454526
Epoch - 280 Train-Loss : 0.18882025529940924 Valid-Loss : 2.331650733947754
unweighted_accuracy : 0.47992351816443596 weighted_accuracy : 0.42943105671296267
Epoch - 285 Train-Loss : 0.1749906837940216 Valid-Loss : 2.4118499755859375
unweighted_accuracy : 0.4340344168260038 weighted_accuracy : 0.4196607987642005
Epoch - 290 Train-Loss : 0.15700294574101767 Valid-Loss : 2.456967830657959
unweighted_accuracy : 0.47418738049713194 weighted_accuracy : 0.43513515798576585
Epoch - 295 Train-Loss : 0.15913129349549612 Valid-Loss : 2.5600662231445312
unweighted_accuracy : 0.47036328871892924 weighted_accuracy : 0.4304163484127558
Epoch - 300 Train-Loss : 0.15626291185617447 Valid-Loss 

/anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [40]:
text_train_loader = DataLoader(train_text[1], batch_size=1000, shuffle=False)
text_valid_loader = DataLoader(valid_text[1], batch_size=1000, shuffle=False)
comment="Text validation : {}".format(1)
t_loss, t_uwa, t_wa = validate_text(text_train_loader, text_valid_loader, vocab_size, 50, 64, pretrained_weights, comment)

t_losses.append(t_loss)
t_uwas.append(t_uwa)
t_was.append(t_wa)

Training Text Model
Epoch - 5 Train-Loss : 1.2688506444295247 Valid-Loss : 1.1891313791275024
unweighted_accuracy : 0.4102079395085066 weighted_accuracy : 0.4102079395085066
Epoch - 10 Train-Loss : 1.2730892896652222 Valid-Loss : 1.2046594619750977
unweighted_accuracy : 0.4102079395085066 weighted_accuracy : 0.4102079395085066
Epoch - 15 Train-Loss : 1.2679523229599 Valid-Loss : 1.4244393110275269
unweighted_accuracy : 0.31568998109640833 weighted_accuracy : 0.15345630308846003
Epoch - 20 Train-Loss : 1.2722065448760986 Valid-Loss : 1.199858546257019
unweighted_accuracy : 0.4102079395085066 weighted_accuracy : 0.4102079395085066
Epoch - 25 Train-Loss : 1.2716678778330486 Valid-Loss : 1.197521448135376
unweighted_accuracy : 0.4102079395085066 weighted_accuracy : 0.4102079395085066
Epoch - 30 Train-Loss : 1.2698241869608562 Valid-Loss : 1.19846773147583
unweighted_accuracy : 0.4102079395085066 weighted_accuracy : 0.4102079395085066
Epoch - 35 Train-Loss : 1.268721103668213 Valid-Loss : 1

Epoch - 270 Train-Loss : 0.20494285225868225 Valid-Loss : 2.3988637924194336
unweighted_accuracy : 0.5236294896030246 weighted_accuracy : 0.4458747262995702
Epoch - 275 Train-Loss : 0.19212614496548971 Valid-Loss : 2.370833396911621
unweighted_accuracy : 0.5293005671077504 weighted_accuracy : 0.4525442067993227
Epoch - 280 Train-Loss : 0.19096198678016663 Valid-Loss : 2.396437168121338
unweighted_accuracy : 0.5425330812854442 weighted_accuracy : 0.4638797038568891
Epoch - 285 Train-Loss : 0.18809436758359274 Valid-Loss : 2.4442970752716064
unweighted_accuracy : 0.5463137996219282 weighted_accuracy : 0.4609618628109766
Epoch - 290 Train-Loss : 0.18185112377007803 Valid-Loss : 2.414764881134033
unweighted_accuracy : 0.5274102079395085 weighted_accuracy : 0.44920904890612795
Epoch - 295 Train-Loss : 0.18395421405633292 Valid-Loss : 2.4622654914855957
unweighted_accuracy : 0.5387523629489603 weighted_accuracy : 0.4660573470168148
Epoch - 300 Train-Loss : 0.18060087660948435 Valid-Loss : 2.

In [41]:
text_train_loader = DataLoader(train_text[2], batch_size=1000, shuffle=False)
text_valid_loader = DataLoader(valid_text[2], batch_size=1000, shuffle=False)
comment="Text validation : {}".format(2)
t_loss, t_uwa, t_wa = validate_text(text_train_loader, text_valid_loader, vocab_size, 50, 64, pretrained_weights, comment)

t_losses.append(t_loss)
t_uwas.append(t_uwa)
t_was.append(t_wa)

Training Text Model
Epoch - 5 Train-Loss : 1.2205537954966228 Valid-Loss : 1.493313193321228
unweighted_accuracy : 0.31369426751592355 weighted_accuracy : 0.31369426751592355
Epoch - 10 Train-Loss : 1.221978227297465 Valid-Loss : 1.4391615390777588
unweighted_accuracy : 0.31369426751592355 weighted_accuracy : 0.31369426751592355
Epoch - 15 Train-Loss : 1.2211787303288777 Valid-Loss : 1.456871509552002
unweighted_accuracy : 0.31369426751592355 weighted_accuracy : 0.31369426751592355
Epoch - 20 Train-Loss : 1.2216686010360718 Valid-Loss : 1.4578930139541626
unweighted_accuracy : 0.31369426751592355 weighted_accuracy : 0.31369426751592355
Epoch - 25 Train-Loss : 1.220909595489502 Valid-Loss : 1.4569162130355835
unweighted_accuracy : 0.31369426751592355 weighted_accuracy : 0.31369426751592355
Epoch - 30 Train-Loss : 1.2193565766016643 Valid-Loss : 1.4564902782440186
unweighted_accuracy : 0.31369426751592355 weighted_accuracy : 0.31369426751592355
Epoch - 35 Train-Loss : 1.2509472370147705 

Epoch - 270 Train-Loss : 0.18473405639330545 Valid-Loss : 2.6134984493255615
unweighted_accuracy : 0.48089171974522293 weighted_accuracy : 0.4367530345471522
Epoch - 275 Train-Loss : 0.1691186601916949 Valid-Loss : 2.4669904708862305
unweighted_accuracy : 0.47770700636942676 weighted_accuracy : 0.45568635666365614
Epoch - 280 Train-Loss : 0.16401877999305725 Valid-Loss : 2.488884687423706
unweighted_accuracy : 0.46337579617834396 weighted_accuracy : 0.4469463322861408
Epoch - 285 Train-Loss : 0.15421398480733237 Valid-Loss : 2.709904432296753
unweighted_accuracy : 0.47770700636942676 weighted_accuracy : 0.45699766777195083
Epoch - 290 Train-Loss : 0.16957303881645203 Valid-Loss : 2.662468671798706
unweighted_accuracy : 0.47611464968152867 weighted_accuracy : 0.4371794871794872
Epoch - 295 Train-Loss : 0.16663318872451782 Valid-Loss : 2.580683708190918
unweighted_accuracy : 0.46496815286624205 weighted_accuracy : 0.4411930598408371
Epoch - 300 Train-Loss : 0.1664936045805613 Valid-Loss 

In [42]:
text_train_loader = DataLoader(train_text[3], batch_size=1000, shuffle=False)
text_valid_loader = DataLoader(valid_text[3], batch_size=1000, shuffle=False)
comment="Text validation : {}".format(3)
t_loss, t_uwa, t_wa = validate_text(text_train_loader, text_valid_loader, vocab_size, 50, 64, pretrained_weights, comment)

t_losses.append(t_loss)
t_uwas.append(t_uwa)
t_was.append(t_wa)

Training Text Model
Epoch - 5 Train-Loss : 1.2502099672953289 Valid-Loss : 1.3604512214660645
unweighted_accuracy : 0.3295880149812734 weighted_accuracy : 0.3295880149812734
Epoch - 10 Train-Loss : 1.2485563357671101 Valid-Loss : 1.3529784679412842
unweighted_accuracy : 0.3295880149812734 weighted_accuracy : 0.3295880149812734
Epoch - 15 Train-Loss : 1.246622363726298 Valid-Loss : 1.3573945760726929
unweighted_accuracy : 0.3295880149812734 weighted_accuracy : 0.3295880149812734
Epoch - 20 Train-Loss : 1.2460387150446575 Valid-Loss : 1.3495219945907593
unweighted_accuracy : 0.33146067415730335 weighted_accuracy : 0.22180293501048218
Epoch - 25 Train-Loss : 1.2450847625732422 Valid-Loss : 1.3543603420257568
unweighted_accuracy : 0.33146067415730335 weighted_accuracy : 0.332391713747646
Epoch - 30 Train-Loss : 1.2444626490275066 Valid-Loss : 1.3553866147994995
unweighted_accuracy : 0.33146067415730335 weighted_accuracy : 0.332391713747646
Epoch - 35 Train-Loss : 1.2442583640416462 Valid-L

Epoch - 270 Train-Loss : 0.12728859980901083 Valid-Loss : 2.5325253009796143
unweighted_accuracy : 0.5093632958801498 weighted_accuracy : 0.47901915725560895
Epoch - 275 Train-Loss : 0.13324010372161865 Valid-Loss : 2.620513677597046
unweighted_accuracy : 0.5224719101123596 weighted_accuracy : 0.4832224242060308
Epoch - 280 Train-Loss : 0.14481928944587708 Valid-Loss : 2.7182071208953857
unweighted_accuracy : 0.5131086142322098 weighted_accuracy : 0.48563645468470495
Epoch - 285 Train-Loss : 0.13479936867952347 Valid-Loss : 2.4874484539031982
unweighted_accuracy : 0.5131086142322098 weighted_accuracy : 0.4860992556983364
Epoch - 290 Train-Loss : 0.13693790634473166 Valid-Loss : 2.574989080429077
unweighted_accuracy : 0.5355805243445693 weighted_accuracy : 0.5139700700576213
Epoch - 295 Train-Loss : 0.12911691019932428 Valid-Loss : 2.465425729751587
unweighted_accuracy : 0.5374531835205992 weighted_accuracy : 0.5068866738334838
Epoch - 300 Train-Loss : 0.1235200713078181 Valid-Loss : 2.

In [43]:
text_train_loader = DataLoader(train_text[4], batch_size=1000, shuffle=False)
text_valid_loader = DataLoader(valid_text[4], batch_size=1000, shuffle=False)
comment="Text validation : {}".format(4)
t_loss, t_uwa, t_wa = validate_text(text_train_loader, text_valid_loader, vocab_size, 50, 64, pretrained_weights, comment)

t_losses.append(t_loss)
t_uwas.append(t_uwa)
t_was.append(t_wa)

Training Text Model
Epoch - 5 Train-Loss : 1.3014346361160278 Valid-Loss : 1.2054779529571533
unweighted_accuracy : 0.3895747599451303 weighted_accuracy : 0.1953232462173315
Epoch - 10 Train-Loss : 1.2969671090443928 Valid-Loss : 1.225237488746643
unweighted_accuracy : 0.38820301783264743 weighted_accuracy : 0.1966804979253112
Epoch - 15 Train-Loss : 1.295925259590149 Valid-Loss : 1.2184544801712036
unweighted_accuracy : 0.39231824417009603 weighted_accuracy : 0.4639080459770115
Epoch - 20 Train-Loss : 1.2938182751337688 Valid-Loss : 1.2201236486434937
unweighted_accuracy : 0.38820301783264743 weighted_accuracy : 0.21334716459197786
Epoch - 25 Train-Loss : 1.2948696215947468 Valid-Loss : 1.2193821668624878
unweighted_accuracy : 0.38820301783264743 weighted_accuracy : 0.1966804979253112
Epoch - 30 Train-Loss : 1.294488787651062 Valid-Loss : 1.2226511240005493
unweighted_accuracy : 0.38820301783264743 weighted_accuracy : 0.24112494236975565
Epoch - 35 Train-Loss : 1.2940397262573242 Vali

Epoch - 270 Train-Loss : 0.17558061083157858 Valid-Loss : 2.3676323890686035
unweighted_accuracy : 0.48148148148148145 weighted_accuracy : 0.4282874196510561
Epoch - 275 Train-Loss : 0.23907069861888885 Valid-Loss : 2.2795894145965576
unweighted_accuracy : 0.5075445816186557 weighted_accuracy : 0.4358174159186753
Epoch - 280 Train-Loss : 0.21635495126247406 Valid-Loss : 2.2756118774414062
unweighted_accuracy : 0.4842249657064472 weighted_accuracy : 0.40847647240454926
Epoch - 285 Train-Loss : 0.20182236532370249 Valid-Loss : 2.4388794898986816
unweighted_accuracy : 0.4869684499314129 weighted_accuracy : 0.4248137674901519
Epoch - 290 Train-Loss : 0.19484174251556396 Valid-Loss : 2.3866307735443115
unweighted_accuracy : 0.4828532235939643 weighted_accuracy : 0.41900296198223264
Epoch - 295 Train-Loss : 0.20105614264806113 Valid-Loss : 2.498678684234619
unweighted_accuracy : 0.4842249657064472 weighted_accuracy : 0.43256696298457975
Epoch - 300 Train-Loss : 0.17783045768737793 Valid-Loss

In [44]:
from statistics import mean 
mean(t_losses), mean(t_uwas), mean(t_was)

(2.547297811508179, 0.4965194328310637, 0.4547437911222483)